## TASK: Detect faces in video feed from live webcam

### Steps
- Import the libraries
- Collect pre-trained HAAR Cascade filters (xml)
- Create a Cascade Classifier with those filters
- Capture video feed from webcam
- Display the video frame
- Convert the RGB frame into GrayScale
- Detect faces with the Cascade Classifier
- Find Face (Region of Interest)
- Draw rectangle around face
- Detect eyes with the Eyes Cascade classifier
- Draw circles for detected eyes
- Show the detected face region with detected eyes

### Putting it all together

In [27]:
import cv2
import os

cascPathface = os.path.dirname(
    cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
cascPatheyes = os.path.dirname(
    cv2.__file__) + "/data/haarcascade_eye_tree_eyeglasses.xml"

# Get image and shape
image = cv2.imread("dogface.png")
o_img_h, o_img_w, o_color = image.shape

# convert to gray
image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Create mask and inverse mask
ret, o_img_mask = cv2.threshold(image_gray, 10, 255, cv2.THRESH_BINARY_INV)
o_img_mask_inv = cv2.bitwise_not(o_img_mask)


faceCascade = cv2.CascadeClassifier(cascPathface)
eyeCascade = cv2.CascadeClassifier(cascPatheyes)

video_capture = cv2.VideoCapture(0)
ret, frame = video_capture.read()
frame_h, frame_w = frame.shape[:2]

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray,
                                         scaleFactor=1.3,
                                         minNeighbors=5,
                                         flags=cv2.CASCADE_SCALE_IMAGE)
    
    for (x,y,w,h) in faces:
        face_w = w
        face_h = h
        face_x1 = x
        face_x2 = face_x1 + face_w
        face_y1 = y
        face_y2 = face_y1 + face_h

        dog_width = int(1.5 * face_w)
        dog_height = int(1.5 * face_h)

        dog_x1 = face_x2 - int(face_w / 2) - int(dog_width / 2)
        dog_x2 = dog_x1 + dog_width
        dog_y1 = int(face_y1 / 1.5)
        dog_y2 = dog_y1 + dog_height 

        if dog_x1 < 0:
            dog_x1 = 0
        if dog_y1 < 0:
            dog_y1 = 0
        if dog_x2 > frame_w:
            dog_x2 = frame_w
        if dog_y2 > frame_h:
            dog_y2 = frame_h

        dog_width = dog_x2 - dog_x1
        dog_height = dog_y2 - dog_y1

        #resize dog_img to fit on face
        dog = cv2.resize(image, (dog_width, dog_height), interpolation = cv2.INTER_AREA)
        mask = cv2.resize(o_img_mask, (dog_width, dog_height), interpolation = cv2.INTER_AREA)
        mask_inv = cv2.resize(o_img_mask_inv, (dog_width, dog_height), interpolation = cv2.INTER_AREA)

        #take ROI for dog_img from background that is equal to size of dog_img image
        roi = frame[dog_y1:dog_y2, dog_x1:dog_x2]

        #original image in background (bg) where dog_img is not
        roi_bg = cv2.bitwise_and(roi, roi, mask = mask)
        roi_fg = cv2.bitwise_and(dog, dog, mask = mask_inv)
        dst = cv2.add(roi_bg,roi_fg)
        
        frame[dog_y1:dog_y2, dog_x1:dog_x2] = dst

        # Display the resulting frame
    cv2.imshow('Face Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video_capture.release()
cv2.destroyAllWindows()